In [1]:
import numpy as np
import pandas as pd

1. Читаем из файла:

In [2]:
df = pd.read_csv('flats.csv', sep=';',  encoding='cp1251', na_values='?')

2. Приводим столбец 'Цена' к числовому типу:

In [3]:
df['Цена,руб'] = df['Цена,руб'].replace(to_replace=r'\s+', value='',regex=True).astype('int64')

3. Удаляем дубликаты квартир:

In [4]:
df = df.sort_values(by='Цена,руб')
df = df.drop_duplicates(subset=['Улица', '№ дома', 'Дом', 'Площадь'], keep = 'first')
df = df.sort_index()

4. Разбиваем столбец Дом на следующие столбцы:

In [5]:
df.insert(6, 'Этаж', '0') #этаж на котором квартира
df.insert(7, 'Количество этажей', '0') #количество этажей в доме
df.insert(8, 'Первый этаж', '0' ) #да = 1, нет = 0
df.insert(9, 'Крайний этаж', '0') #первый или последний = 1,нет = 0
df.insert(10, 'Отношение', '0') #отношение этажа к количеству этажей
df.insert(11, 'Тип дома', '0') #тип дома Б - блочный П - панельный М - монолит М-К - монолит-кирпич К - кирпич

In [6]:
#Получаем первую часть до разделителя / - это этаж
df['Этаж'] = df['Дом'].str.split('/').str.get(0).replace('\\?', np.nan, regex=True)

In [7]:
#Получаем вторую часть после разделителя /, затем получаем первую часть после разделителя ' ' - это количество этажей в доме
df['Количество этажей'] = df['Дом'].str.split('/').str.get(1).str.split(' ').str.get(0).replace('\\?', np.nan, regex=True)

In [8]:
#Получаем этаж, нужно получить первый этаж - меняем единички на нули, что бы отличать первый этаж от других,
#приводим к булеву значению, инвертируем результат.
df['Первый этаж'] =  ~df['Дом'].str.split('/').str.get(0).replace('\\?', np.nan, regex=True).astype('float')\
.replace(1, 0).astype('bool')

In [9]:
#Если количество этажей совпадает с этажом или это первый этаж, то это крайний этаж
df['Крайний этаж'] = np.where((df['Количество этажей'] == df['Этаж']) | df['Первый этаж'] , True, False)

In [10]:
#Просто делим этаж на количество этажей
df['Отношение'] = np.where(df['Этаж'].isnull() | df['Количество этажей'].isnull(), np.nan,\
                           (df['Этаж'].astype('float') /df['Количество этажей'].astype('float')).round(3))

In [11]:
#Получаем тип дома
df['Тип дома'] = df['Дом'].str.split(' ').str.get(1).replace('\\?', np.nan, regex=True)

5. Анализируем дома в зависимости от типа:

In [15]:
allhouse = df[['Тип дома']]
allhouse.insert(1,'Количество', 1)

In [13]:
#a. находим количество и процент для каждого типа домов
allhouse = allhouse.fillna('NaN').groupby('Тип дома').sum()
alz = allhouse['Количество'].sum()
allhouse['Проценты'] = allhouse['Количество'].apply(lambda x: round(float(x / alz) * 100, 2))
allhouse

,Количество,Проценты
Тип дома,,
NaN,1375,15.57
Б,397,4.50
Ж-б,1,0.01
К,1979,22.41
М,1963,22.23
М-К,613,6.94
П,2150,24.35
Стал.,350,3.96
Шлакоблок,1,0.01


In [433]:
#б. убираем типы с одной записью, добавляем их в NaN
allhouse = allhouse.drop(columns='Проценты')
cnt = allhouse.loc[allhouse['Количество'] == 1, 'Количество'].sum()  
allhouse = allhouse[allhouse['Количество'] != 1]
allhouse.at['NaN', 'Количество'] = allhouse.at['NaN', 'Количество'] + cnt

In [434]:
#пересчитываем процент
allhouse['Проценты'] = allhouse['Количество'].apply(lambda x: round(float(x / alz) * 100, 2))
allhouse

,Количество,Проценты
Тип дома,,
NaN,1377,15.60
Б,397,4.50
К,1979,22.41
М,1963,22.23
М-К,613,6.94
П,2150,24.35
Стал.,350,3.96


In [435]:
# Объединение Б и П, М и М-К, пересчитываем проценты
allhouse.index = allhouse.index.to_series().replace({'П':'Б-П', 'Б':'Б-П', 'М':'М-К'}) 

In [436]:
allhouse = allhouse.groupby(allhouse.index, sort=False).sum()
allhouse['Проценты'] = allhouse['Количество'].apply(lambda x: round(float(x/alz) * 100, 2))

In [437]:
allhouse

,Количество,Проценты
Тип дома,,
NaN,1377,15.60
Б-П,2547,28.85
К,1979,22.41
М-К,2576,29.18
Стал.,350,3.96
